In [7]:
import numpy as np
import SimpleITK as sitk
import itk
import os
from scipy import signal
import matplotlib.pyplot as plt

In [8]:
def get_list_of_files(base_dir):

    list_of_lists = []
    patients = os.listdir(base_dir)
    for p in patients:
        if p.startswith("mr_train_") and p.endswith(".nii.gz"):
            list_of_lists.append(os.path.join(base_dir, p))
    print("Found %d patients" % len(list_of_lists))
    return list_of_lists


def load_img(dir_img):
    # load SimpleITK Image
    img_sitk = sitk.ReadImage(dir_img)

    # get pixel arrays from SimpleITK images
    img_npy = sitk.GetArrayFromImage(img_sitk)

    # get some metadata
    spacing = img_sitk.GetSpacing()
    # the spacing returned by SimpleITK is in inverse order relative to the numpy array we receive. If we wanted to
    # resample the data and if the spacing was not isotropic (in BraTS all cases have already been resampled to 1x1x1mm
    # by the organizers) then we need to pay attention here. Therefore we bring the spacing into the correct order so
    # that spacing[0] actually corresponds to the spacing of the first axis of the numpy array
    spacing = np.array(spacing)[::-1]
    direction = img_sitk.GetDirection()
    origin = img_sitk.GetOrigin()

    original_shape = img_npy.shape
    
    metadata = {
    'spacing': spacing,
    'direction': direction,
    'origin': origin,
    'original_shape': original_shape
    }
        
    print("Pixel Type    {}".format(img_sitk.GetPixelID()))
    print("Size          {}".format(img_sitk.GetSize()))
    print("Origin        {}".format(origin))
    print("Spacing       {}".format(spacing))
    print("Direction     {}".format(direction))    
    
    return img_sitk, img_npy, metadata  



def get_center(img):
    """
    This function returns the physical center point of a 3d sitk image
    :param img: The sitk image we are trying to find the center of
    :return: The physical center point of the image
    """
    width, height, depth = img.GetSize()
    
    return img.TransformIndexToPhysicalPoint((int(np.ceil(width/2)),
                                          int(np.ceil(height/2)),
                                          int(np.ceil(depth/2))))


def rotate_img(img_sitk, transform, is_label, theta_x, theta_y, theta_z):
    
#     new_transform = sitk.AffineTransform(transform)
#     matrix = np.array(transform.GetMatrix()).reshape((3,3))
#     radians = -np.pi * theta_x / 180.
#     rotation = np.array([[np.sin(radians), np.cos(radians), 0], 
#                         [0, 0, 1],
#                         [np.cos(radians), -np.sin(radians), 0]] )  # https://stackoverflow.com/questions/9187387/3d-rotation-on-image
#     new_matrix = np.dot(rotation, matrix)
#     new_transform.SetMatrix(new_matrix.ravel())
    
    theta_x = np.deg2rad(theta_x)
    theta_y = np.deg2rad(theta_y)
    theta_z = np.deg2rad(theta_z)
    
    new_transform = sitk.Euler3DTransform(get_center(img_sitk), theta_x, theta_y, theta_z, (0, 0, 0))
    image_center = get_center(img_sitk)
    new_transform.SetCenter(image_center)
    new_transform.SetRotation(theta_x, theta_y, theta_z)  
    
    # Resample
    reference_image = img_sitk
    
    if is_label:
        interpolator = sitk.sitkNearestNeighbor
    else:
        interpolator = sitk.sitkBSpline # sitkCosineWindowedSinc    

    default_value = 0
    resampled = sitk.Resample(img_sitk, reference_image, new_transform,
                         interpolator, default_value)  
    npy_img = sitk.GetArrayFromImage(resampled)
    
    return npy_img, resampled


def reorient_to_rai(image):
    filter = itk.OrientImageFilter.New(image)
    filter.UseImageDirectionOn()
    filter.SetInput(image)
    m = itk.Matrix[itk.D, 3, 3]()
    m.SetIdentity()
    filter.SetDesiredCoordinateDirection(m)
    filter.Update()
    return filter.GetOutput()


def save_as_nii(img_npy, img_name, metadata):
    sitk_image = sitk.GetImageFromArray(img_npy)
    sitk_image.SetDirection(metadata['direction'])
    sitk_image.SetOrigin(metadata['origin'])
    # remember to revert spacing back to sitk order again
    sitk_image.SetSpacing(tuple(metadata['spacing'][[1, 2, 0]]))
    sitk.WriteImage(sitk_image, img_name)   

In [9]:
base_dir = '/usr/not-backed-up2/scsad/DL/MedicalDataAugmentationTool/bin/experiments/semantic_segmentation/mmwhs/TODO_mr'
list_of_files = get_list_of_files(base_dir)
dir_img = list_of_files[38]
dir_label = list_of_files[39]
print(dir_img)
print(dir_label)

Found 40 patients
/usr/not-backed-up2/scsad/DL/MedicalDataAugmentationTool/bin/experiments/semantic_segmentation/mmwhs/TODO_mr/mr_train_1020_image.nii.gz
/usr/not-backed-up2/scsad/DL/MedicalDataAugmentationTool/bin/experiments/semantic_segmentation/mmwhs/TODO_mr/mr_train_1020_label.nii.gz


In [10]:
img_sitk, img_npy, metadata_img = load_img(dir_img)
label_sitk, label_npy, metadata_label = load_img(dir_label)

Pixel Type    2
Size          (288, 288, 135)
Origin        (134.70086669921875, -172.93592834472656, -103.38882446289062)
Spacing       [1.04999995 0.97222221 0.97222221]
Direction     (-0.0, 0.0, -1.0, 1.0, -0.0, 0.0, 0.0, 1.0, -0.0)
Pixel Type    3
Size          (288, 288, 135)
Origin        (134.70086669921875, -172.93592834472656, -103.38882446289062)
Spacing       [1.04999995 0.97222221 0.97222221]
Direction     (-0.0, 0.0, -1.0, 1.0, -0.0, 0.0, 0.0, 1.0, -0.0)


In [8]:
# Rotate image
theta_x, theta_y, theta_z = 0, 30, 0
affine_ro = sitk.AffineTransform(3)
npy_rotated, rotated_image = rotate_img(img_sitk, affine_ro, False, theta_x, theta_y, theta_z) # https://stackoverflow.com/questions/56171643/simpleitk-rotation-of-mri-image
#reorientedToRAI = reorient_to_rai(rotated_image)
sitk.WriteImage(rotated_image, 'rotation_X' + str(theta_x) + 'Y' + str(theta_y) + 'Z' + str(theta_z) + '_' + dir_img.split('/')[11:][0][0:-7] + '.nii.gz', True) 

In [9]:
# Rotate label
affine_ro = sitk.AffineTransform(3)
npy_rotated_label, rotated_label = rotate_img(label_sitk, affine_ro, True, theta_x, theta_y, theta_z) 
#reorientedToRAI_label = reorient_to_rai(rotated_label)
sitk.WriteImage(rotated_label, 'rotation_X' + str(theta_x) + 'Y' + str(theta_y) + 'Z' + str(theta_z) + '_' + dir_img.split('/')[11:][0][0:-7] + '.nii.gz', True) 

In [15]:
# Downsampling image. 

# Check how to reorient the image to RAI and then downsample it to only 16 slices!
# Do the same for the label!

downsampled_img = np.array([npy_rotated[i*10,:,:] for i in range(0, 13)],dtype=np.float32)
    
print(downsampled_img.shape)   
print(metadata_img['spacing'])
save_as_nii(downsampled_img, 'downsample.nii.gz', metadata_img)

(13, 288, 288)
[1.04999995 0.97222221 0.97222221]


In [16]:
# Downsampling label 

downsampled_label = np.array([npy_rotated_label[i*10,:,:] for i in range(0, 13)],dtype=np.float32)
save_as_nii(downsampled_label, 'downsample_label.nii.gz', metadata_label)

In [ ]:
print(img_npy.shape)
print(label_npy.shape)

In [ ]:
for i in range(0,10):
    idx = i*2+65
    plt.imshow(rotated_image[:,idx,:].T, cmap='gray')
    plt.show()

In [ ]:
# We need to have the max resolution of these images in sagital view
# As the sagital view we have from Deaglan images are sagital view with slight inclination,
# we could first rotate training images to have them very similar to SPASM images


# To do this, we need to inclinate the images from the sagital view.
# Take the sagital view images that cover the whole heart

# To rescale images: https://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html

# How can we expand the physical space??